In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
class NN(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(28 * 28, 256)  # First fully connected layer
        self.fc2 = nn.Linear(256, 64)       # Second fully connected layer
        self.out = nn.Linear(64, 10)        # Output layer

    def forward(self, x):
        x = x.view(-1, 28 * 28)  # Flatten the input tensor to match the input size of fc1
        x = F.tanh(self.fc1(x))  # Apply tanh activation to the output of fc1
        x = F.relu(self.fc2(x))  # Apply ReLU activation to the output of fc2
        x = self.out(x)          # Compute the final output
        return x


In [ ]:
model = NN()
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader


In [ ]:
transform = transforms.Compose([
    transforms.ToTensor(),  # Convert images to PyTorch tensors
    transforms.Normalize((0.1307,), (0.3081,))  # Normalize with mean and std of MNIST
])

In [ ]:
# Load the training dataset
train_dataset = torchvision.datasets.FashionMNIST(
    root='./data',       # Directory to store the dataset
    train=True,          # Specifies that this is the training dataset
    download=True,       # Download the dataset if not already available
    transform=transform  # Apply the transformations defined above
)

# Load the test dataset
test_dataset = torchvision.datasets.FashionMNIST(
    root='./data',       # Directory to store the dataset
    train=False,         # Specifies that this is the test dataset
    download=True,       # Download the dataset if not already available
    transform=transform  # Apply the transformations defined above
)


100%|██████████| 26421880/26421880 [00:03<00:00, 8494142.62it/s] 


Extracting ./data/FashionMNIST/raw/train-images-idx3-ubyte.gz to ./data/FashionMNIST/raw



100%|██████████| 29515/29515 [00:00<00:00, 169675.70it/s]


Extracting ./data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to ./data/FashionMNIST/raw



100%|██████████| 4422102/4422102 [00:01<00:00, 3148858.09it/s]


Extracting ./data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to ./data/FashionMNIST/raw



100%|██████████| 5148/5148 [00:00<00:00, 6253193.45it/s]

Extracting ./data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/FashionMNIST/raw



In [ ]:
# Data loader for the training set
train_loader = DataLoader(
    dataset=train_dataset,
    batch_size=64,        # Number of samples per batch
    shuffle=True          # Shuffle the data at every epoch
)

# Data loader for the test set
test_loader = DataLoader(
    dataset=test_dataset,
    batch_size=1000,      # Number of samples per batch
    shuffle=False         # Do not shuffle the test data
)


In [ ]:
# Example: Iterate through the training data
for batch_idx, (data, target) in enumerate(train_loader):
    # data is a batch of images, target is the corresponding labels
    print(data.shape)  # e.g., torch.Size([64, 1, 28, 28])
    print(target.shape)  # e.g., torch.Size([64])
    break  # Just for demonstration, break after one batch


torch.Size([64, 1, 28, 28])
torch.Size([64])


In [ ]:
import torch.optim as opt
device = torch.device('cuda' if torch.cuda.is_available else 'cpu')
model = NN().to(device)

optimizer = opt.Adam(model.parameters(), lr= 0.001)

loss_func = nn.CrossEntropyLoss()


In [ ]:
def train(epoch):
  model.train()
  for batch_idx, (data, target) in enumerate(train_loader):
    data , target = data.to(device), target.to(device)
    optimizer.zero_grad()
    output = model(data)
    loss = loss_func(output,target)
    loss.backward()
    optimizer.step()
    if batch_idx % 20 ==0:
      print(f'Train Epoch: {epoch} |{batch_idx * len(data)}/ {len(train_dataset)} ({100. * batch_idx / len(train_loader):.0f}%)]\t{loss.item():.6f} ')






In [ ]:
def test():
  model.eval()

  test_loss =0
  correct =0
  with torch.no_grad():
    for data, target in test_loader:
      data, target = data.to(device), target.to(device)
      output = model(data)
      test_loss += loss_func(output,target).item()
      pred = output.argmax(dim =1,keepdim = True)
      correct += pred.eq(target.view_as(pred)).sum().item()
  test_loss /= len(test_dataset)
  #print(f'Test set: Average Loss:{test_loss:.4f},Accuracy:{correct}/{len(test_dataset)} ({100. *correct / len(test_dataset):.0f }%)')
  print(f'Test set: Average Loss: {test_loss:,.4f}, Accuracy: {correct}/{len(test_dataset)} ({100. * correct / len(test_dataset):.0f}%)')




In [ ]:
for epoch in range(1,11):
  train(epoch)
  test()

Train Epoch: 1 |0/ 60000 (0%)]	0.246029 
Train Epoch: 1 |1280/ 60000 (2%)]	0.239396 
Train Epoch: 1 |2560/ 60000 (4%)]	0.268649 
Train Epoch: 1 |3840/ 60000 (6%)]	0.335800 
Train Epoch: 1 |5120/ 60000 (9%)]	0.322957 
Train Epoch: 1 |6400/ 60000 (11%)]	0.348036 
Train Epoch: 1 |7680/ 60000 (13%)]	0.410276 
Train Epoch: 1 |8960/ 60000 (15%)]	0.276219 
Train Epoch: 1 |10240/ 60000 (17%)]	0.196444 
Train Epoch: 1 |11520/ 60000 (19%)]	0.382196 
Train Epoch: 1 |12800/ 60000 (21%)]	0.213351 
Train Epoch: 1 |14080/ 60000 (23%)]	0.272018 
Train Epoch: 1 |15360/ 60000 (26%)]	0.143004 
Train Epoch: 1 |16640/ 60000 (28%)]	0.147555 
Train Epoch: 1 |17920/ 60000 (30%)]	0.243269 
Train Epoch: 1 |19200/ 60000 (32%)]	0.202357 
Train Epoch: 1 |20480/ 60000 (34%)]	0.223693 
Train Epoch: 1 |21760/ 60000 (36%)]	0.288553 
Train Epoch: 1 |23040/ 60000 (38%)]	0.448417 
Train Epoch: 1 |24320/ 60000 (41%)]	0.200891 
Train Epoch: 1 |25600/ 60000 (43%)]	0.410369 
Train Epoch: 1 |26880/ 60000 (45%)]	0.160505 
Trai